# Imports and NLP


In [8]:
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  !pip install fuzzywuzzy[speedup]
else:
  print("Not running in colab")

Running on CoLab
     |████████████████████████████████| 51kB 5.8MB/s 
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144794 sha256=88301cccde91aa1fb171bf0fbd39ecf57911e7dfb3eea5b9b038c7de357feed2
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-levenshtein


In [2]:
!pip install colab_everything

     |████████████████████████████████| 133kB 16.9MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.1-cp36-none-any.whl size=18822 sha256=f670359758a8d372bfdc767583f787b5c5c0afae9bcdf232b7675ecf39660746
  Stored in directory: /root/.cache/pip/wheels/94/01/05/d39efb8f6b40a411354b4168ca9dda99e6f8d586e458e97551
Successfully built pyngrok
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [4]:
!pip install fastapi

     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
     |████████████████████████████████| 9.2MB 17.8MB/s 


In [6]:
!pip install flask_ngrok

In [56]:
import ast
import itertools
import numpy as np
import pandas as pd 
import pickle
import re
import spacy
import spacy.cli
import time
from colab_everything import ColabCustom
# from colab_everything import ColabFastapi
from fastapi import FastAPI, Request
from fastapi.templating import Jinja2Templates
from flask import Flask
from flask_ngrok import run_with_ngrok
from fuzzywuzzy import process
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler
from time import process_time, gmtime, strftime

In [10]:
%load_ext google.colab.data_table

In [11]:
spacy.cli.download("en_core_web_lg")
nlp = spacy.load("en_core_web_lg")

⚠ Skipping model package dependencies and setting `--no-deps`. You
don't seem to have the spaCy package itself installed (maybe because you've
built from source?), so installing the model dependencies would cause spaCy to
be downloaded, which probably isn't what you want. If the model package has
other dependencies, you'll have to install them manually.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


# Examine Data

In [30]:
# @title Retrieve Data

url = "https://raw.githubusercontent.com/vjmiyagi/DS_BuildWeek_Dec_2020/main/csv/data.csv"
df_main = pd.read_csv(url)
df_artist = pd.read_csv("https://raw.githubusercontent.com/vjmiyagi/DS_BuildWeek_Dec_2020/main/csv/data_by_artist.csv")
df_genre = pd.read_csv("https://raw.githubusercontent.com/vjmiyagi/DS_BuildWeek_Dec_2020/main/csv/data_by_genres.csv")
df_year = pd.read_csv("https://raw.githubusercontent.com/vjmiyagi/DS_BuildWeek_Dec_2020/main/csv/data_by_year.csv")
df_w = pd.read_csv("https://raw.githubusercontent.com/vjmiyagi/DS_BuildWeek_Dec_2020/main/csv/data_w_genres.csv")

new_row = {"valence": 0.404, "year": 1962, "acousticness": 0.707, 
            "artists": "['Kyu Sakamoto']", "danceability": 0.415, 
            "duration_ms": 189520, "energy": 0.359, "explicit": 1, 
            "id": "6meIeOX3DHdaCnaNw67abE", "instrumentalness": 0.0000653,
            "key": 7, "liveness": 0.319, "loudness": -11.094, "mode": 1, 
            "name": "上を向いて歩こう Ue Wo Muite Arukou Sukiyaki", "popularity": 50,
            "release_date": "2010-01-01", "speechiness": 0.0334, "tempo": 146.029}

df_main = df_main.append(new_row, ignore_index=True)


In [31]:
#@title Heads or Tails  { run: "auto" }
show = "Heads" #@param ["Heads", "Tails"]
from __future__ import print_function

from google.colab import widgets
from google.colab import output


a = df_main.head()
b = df_main.tail()
c = df_artist.head()
d = df_artist.tail()
e = df_genre.head()
f = df_genre.tail()
g = df_year.head()
h = df_year.tail()
i = df_year.head()
j = df_year.tail()

heads = [a, c, e, f, i]
tails = [b, d, f, h, j]

def TB():
    tb = widgets.TabBar(["Data", "Artist", "Genre", "Year", "Genre with Artist"])
    l_tab = show
    for x in range(5):
        with tb.output_to(x):
            if l_tab == "Heads":
                print(heads[x])
            elif l_tab == "Tails":
                print(tails[x])

dash = '=' * 81
print(dash)
t7 = "Shapes"
print(f'{t7:^81}')
print()
s1, s2, s3, s4, s5 = ("Main", "Artist", "Genre",
                              "Year", "Genre with Artist")
print(f'{s1:^16}{s2:^16}{s3:^16}{s4:^16}{s5:^16}')
t1, t2, t3, t4, t5 = (str(df_main.shape), str(df_artist.shape), 
                          str(df_genre.shape), str(df_year.shape),
                          str(df_w.shape))

print(f'{t1:^16}{t2:^16}{t3:^16}{t4:^16}{t5:^16}')
print()
print(dash)





TB()

                                     Shapes                                      

      Main           Artist          Genre            Year      Genre with Artist
  (170654, 19)    (28680, 15)      (2973, 14)      (100, 14)      (28680, 16)   



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

   mode  year  acousticness  ...   valence  popularity  key
0     1  1921      0.886896  ...  0.379327    0.653333    2
1     1  1922      0.938592  ...  0.535549    0.140845   10
2     1  1923      0.957247  ...  0.625492    5.389189    0
3     1  1924      0.940200  ...  0.663725    0.661017   10
4     1  1925      0.962607  ...  0.621929    2.604317    5

[5 rows x 14 columns]


<IPython.core.display.Javascript object>

In [32]:
#@title Heads or Tails ... a closer look  { run: "auto" }
show = "Heads" #@param ["Heads", "Tails"]
from __future__ import print_function

from google.colab import widgets
from google.colab import output

z1 = len(df_main) - 1
z2 = len(df_artist) - 1
z3 = len(df_genre) - 1
z4 = len(df_year) - 1
z5 = len(df_w) - 1
a = df_main.loc[0]
b = df_main.loc[z1]
c = df_artist.loc[0]
d = df_artist.loc[z2]
e = df_genre.loc[0]
f = df_genre.loc[z3]
g = df_year.loc[0]
h = df_year.loc[z4]
i = df_w.loc[0]
j = df_w.loc[z5]

heads = [a, c, e, f, i]
tails = [b, d, f, h, j]

def TB():
    tb = widgets.TabBar(["Data", "Artist", "Genre", "Year", "Genre with Artist"])
    l_tab = show
    for x in range(5):
        with tb.output_to(x):
            if l_tab == "Heads":
                print(heads[x])
            elif l_tab == "Tails":
                print(tails[x])

dash = '=' * 81
print(dash)
t7 = "Shapes"
print(f'{t7:^81}')
print()
s1, s2, s3, s4, s5 = ("Main", "Artist", "Genre",
                              "Year", "Genre with Artist")
print(f'{s1:^16}{s2:^16}{s3:^16}{s4:^16}{s5:^16}')
t1, t2, t3, t4, t5 = (str(df_main.shape), str(df_artist.shape), 
                          str(df_genre.shape), str(df_year.shape),
                          str(df_w.shape))

print(f'{t1:^16}{t2:^16}{t3:^16}{t4:^16}{t5:^16}')
print()
print(dash)





TB()

                                     Shapes                                      

      Main           Artist          Genre            Year      Genre with Artist
  (170654, 19)    (28680, 15)      (2973, 14)      (100, 14)      (28680, 16)   



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

genres                                ['show tunes']
artists             "Cats" 1981 Original London Cast
acousticness                                0.590111
danceability                                0.467222
duration_ms                                   250319
energy                                      0.394003
instrumentalness                           0.0113999
liveness                                    0.290833
loudness                                     -14.448
speechiness                                 0.210389
tempo                                        117.518
valence                                       0.3895
popularity                                   38.3333
key                                                5
mode                                               1
count                                              9
Name: 0, dtype: object


<IPython.core.display.Javascript object>

In [33]:
#@title Heads or Tails ... a closer look  { run: "auto" }
show = "Tails" #@param ["Heads", "Tails"]
from __future__ import print_function

from google.colab import widgets
from google.colab import output

z1 = len(df_main) - 1
z2 = len(df_artist) - 1
z3 = len(df_genre) - 1
z4 = len(df_year) - 1
z5 = len(df_w) - 1
a = df_main.loc[0]
b = df_main.loc[z1]
c = df_artist.loc[0]
d = df_artist.loc[z2]
e = df_genre.loc[0]
f = df_genre.loc[z3]
g = df_year.loc[0]
h = df_year.loc[z4]
i = df_w.loc[0]
j = df_w.loc[z5]

heads = [a, c, e, f, i]
tails = [b, d, f, h, j]

def TB():
    tb = widgets.TabBar(["Data", "Artist", "Genre", "Year", "Genre with Artist"])
    l_tab = show
    for x in range(5):
        with tb.output_to(x):
            if l_tab == "Heads":
                print(heads[x])
            elif l_tab == "Tails":
                print(tails[x])

dash = '=' * 81
print(dash)
t7 = "Shapes"
print(f'{t7:^81}')
print()
s1, s2, s3, s4, s5 = ("Main", "Artist", "Genre",
                              "Year", "Genre with Artist")
print(f'{s1:^16}{s2:^16}{s3:^16}{s4:^16}{s5:^16}')
t1, t2, t3, t4, t5 = (str(df_main.shape), str(df_artist.shape), 
                          str(df_genre.shape), str(df_year.shape),
                          str(df_w.shape))

print(f'{t1:^16}{t2:^16}{t3:^16}{t4:^16}{t5:^16}')
print()
print(dash)





TB()

                                     Shapes                                      

      Main           Artist          Genre            Year      Genre with Artist
  (170654, 19)    (28680, 15)      (2973, 14)      (100, 14)      (28680, 16)   



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

genres              ['classic korean pop']
artists                                조정현
acousticness                         0.568
danceability                         0.447
duration_ms                         237688
energy                               0.215
instrumentalness                  1.16e-06
liveness                            0.0649
loudness                           -16.478
speechiness                         0.0272
tempo                               71.979
valence                              0.177
popularity                              31
key                                     10
mode                                     1
count                                    2
Name: 28679, dtype: object


<IPython.core.display.Javascript object>

In [34]:
df = df_main
df

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.98200,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.73200,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.96100,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.96700,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.95700,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170649,0.7340,2020,0.20600,['Ashnikko'],0.717,150654,0.753,0,0OStKKAuXlxA0fMH54Qs6E,0.000000,7,0.101,-6.020,1,Halloweenie III: Seven Days,68,2020-10-23,0.0605,137.936
170650,0.6370,2020,0.10100,['MAMAMOO'],0.634,211280,0.858,0,4BZXVFYCb76Q0Klojq4piV,0.000009,4,0.258,-2.226,0,AYA,76,2020-11-03,0.0809,91.688
170651,0.1950,2020,0.00998,['Eminem'],0.671,337147,0.623,1,5SiZJoLXp3WOl3J4C8IK0d,0.000008,2,0.643,-7.161,1,Darkness,70,2020-01-17,0.3080,75.055
170652,0.6420,2020,0.13200,"['KEVVO', 'J Balvin']",0.856,189507,0.721,1,7HmnJHfs0BkFzX4x8j0hkl,0.004710,7,0.182,-4.928,1,Billetes Azules (with J Balvin),74,2020-10-16,0.1080,94.991


In [35]:
df["artists"]

0         ['Sergei Rachmaninoff', 'James Levine', 'Berli...
1                                            ['Dennis Day']
2         ['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...
3                                          ['Frank Parker']
4                                            ['Phil Regan']
                                ...                        
170649                                         ['Ashnikko']
170650                                          ['MAMAMOO']
170651                                           ['Eminem']
170652                                ['KEVVO', 'J Balvin']
170653                                     ['Kyu Sakamoto']
Name: artists, Length: 170654, dtype: object

In [36]:
print("df:", df.shape)
print(df.head(1))
print("df_artist:", df_artist.shape)
print(df_artist.head(1))
print("df_genre:", df_genre.shape)
print(df_genre.head(1))
print("df_year:", df_year.shape)
print(df_year.head(1))
print("df_w:", df_w.shape)
print(df_w.head(1))

df_w.tail(1)

df: (170654, 19)
   valence  year  acousticness  ... release_date  speechiness   tempo
0   0.0594  1921         0.982  ...         1921       0.0366  80.954

[1 rows x 19 columns]
df_artist: (28680, 15)
   mode  count  acousticness  ... valence  popularity  key
0     1      9      0.590111  ...  0.3895   38.333333    5

[1 rows x 15 columns]
df_genre: (2973, 14)
   mode                  genres  acousticness  ...   valence  popularity  key
0     1  21st century classical      0.979333  ...  0.103783   27.833333    6

[1 rows x 14 columns]
df_year: (100, 14)
   mode  year  acousticness  ...   valence  popularity  key
0     1  1921      0.886896  ...  0.379327    0.653333    2

[1 rows x 14 columns]
df_w: (28680, 16)
           genres                           artists  ...  mode  count
0  ['show tunes']  "Cats" 1981 Original London Cast  ...     1      9

[1 rows x 16 columns]


,genres,artists,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,mode,count
28679,['classic korean pop'],조정현,0.568,0.447,237688.0,0.215,0.000001,0.0649,-16.478,0.0272,71.979,0.177,31.0,10,1,2


# Get to work with the main DataFrame, df, and focus on text fields

In [19]:
# Convert string to list
df.artists = df.artists.apply(ast.literal_eval)
# returning a list store in a string, str('[a,g,c]') --> list [a,g,c]
print("Artist Column Type: ", type(df.artists[0]))
df['artists']

Artist Column Type:  <class 'list'>


0         [Sergei Rachmaninoff, James Levine, Berliner P...
1                                              [Dennis Day]
2         [KHP Kridhamardawa Karaton Ngayogyakarta Hadin...
3                                            [Frank Parker]
4                                              [Phil Regan]
                                ...                        
170649                                           [Ashnikko]
170650                                            [MAMAMOO]
170651                                             [Eminem]
170652                                    [KEVVO, J Balvin]
170653                                       [Kyu Sakamoto]
Name: artists, Length: 170654, dtype: object

In [20]:
def lst_join(list_artists):
  """transform from list to string
  to get rid of the [] of the list"""
  Jointed_list = ", ".join(list_artists)
  return Jointed_list

df.artists = df.artists.apply(lst_join)
df[['artists','id']].head()

,artists,id
0,"Sergei Rachmaninoff, James Levine, Berliner Ph...",4BJqT0PrAfrxzMOxytFOIz
1,Dennis Day,7xPhfUan2yNtyFG0cUWkt8
2,KHP Kridhamardawa Karaton Ngayogyakarta Hadini...,1o6I8BglA6ylDMrIELygv1
3,Frank Parker,3ftBPsC5vPBKxYSee08FDH
4,Phil Regan,4d6HGyGT8e121BsdKmw9v6


In [21]:
df['name_artist'] = df['name'] + ", " + df['artists']
df[['name_artist']].head()

,name_artist
0,"Piano Concerto No. 3 in D Minor, Op. 30: III. ..."
1,"Clancy Lowered the Boom, Dennis Day"
2,"Gati Bali, KHP Kridhamardawa Karaton Ngayogyak..."
3,"Danny Boy, Frank Parker"
4,"When Irish Eyes Are Smiling, Phil Regan"


In [24]:
#@title Use name column - VECTORIZE - Results to be df2
# Making computer sense out of all these words
then = time.time() #Time before the operations start

print("Grab a cup of coffee, this is gonna take a while.")
print("Started running:")
print(strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime()))
x = [nlp(str(item)).vector for item in df['name']]
now = time.time() #Time after it finished

z = (now-then)/60
print(f"It took: {z:0.1f} minutes vectorize name column")
df2 = pd.DataFrame(x)
print(df2.shape)
df2.head()


Grab a cup of coffee, this is gonna take a while.
Started running:
Thu, 07 Jan 2021 01:07:19 +0000
It took: 16.3 minutes vectorize name column
(170654, 300)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,-0.085524,0.253920,-0.054556,-0.189623,0.111724,0.037554,0.135672,-0.149358,0.109217,1.118779,-0.276632,0.331278,0.039741,0.078383,-0.120770,0.050645,0.086520,1.198039,-0.143236,-0.051517,-0.000259,0.035743,-0.009834,0.012668,0.097151,0.035742,-0.255916,0.004644,0.321199,0.054373,0.002129,-0.061250,-0.064901,0.003917,0.034910,-0.056363,0.049580,0.034655,0.057413,-0.102587,...,0.132846,0.115589,-0.009819,0.133983,-0.072497,-0.206486,0.130294,-0.143455,0.093379,-0.053187,-0.112625,-0.108820,-0.132623,-0.080294,-0.079285,0.141592,0.157666,-0.098600,-0.076094,0.169554,0.335679,-0.143228,0.181069,-0.275473,-0.093530,0.005273,-0.054336,-0.194938,0.185263,-0.103693,0.162488,-0.078363,-0.120692,-0.042693,0.229962,0.125117,0.003538,-0.036525,-0.188328,0.062255
1,-0.041671,0.133612,-0.130652,-0.013394,0.442913,0.036117,-0.108705,0.197338,0.056023,1.202923,-0.016608,-0.350428,0.353532,0.088840,0.159057,-0.134302,-0.055932,0.653183,-0.166985,0.011920,0.142126,-0.076178,0.257854,-0.217825,-0.113594,-0.075012,-0.276372,0.138515,-0.195980,0.048988,-0.010214,0.151734,0.113735,0.256612,0.123147,-0.162056,-0.176280,0.124145,0.156631,-0.169549,...,-0.092842,-0.015892,-0.131619,0.043470,0.097627,-0.248162,0.150887,-0.116428,-0.189411,0.004541,0.411285,-0.179345,0.138213,0.011857,0.196374,-0.317490,0.018159,0.263573,0.133511,-0.018819,-0.145609,-0.063024,-0.237998,-0.164808,0.140370,0.007817,-0.062990,-0.150875,0.108841,0.074159,-0.417127,0.299880,0.132565,-0.059358,0.004424,-0.158116,-0.307742,0.103382,-0.087310,-0.162832
2,0.315735,0.117667,0.487630,0.186645,0.396040,0.153680,0.288910,-0.456795,0.162340,-1.033880,-0.488075,0.151065,0.015475,0.019775,0.220265,0.214030,0.080650,0.028925,0.714515,0.233120,-0.016040,0.615075,0.076660,0.154880,-0.128703,-0.183765,-0.114554,-0.140250,-0.312595,0.818685,0.308660,-0.606500,-0.078873,-0.102335,-0.665195,-0.305484,0.064665,0.076977,-0.586460,-0.101884,...,0.410140,-0.313229,0.292151,-0.002265,0.025895,-0.077975,0.136851,0.230245,-0.592530,0.056080,-0.882825,-0.337010,0.115729,0.279485,0.023775,0.094330,0.332201,-0.082320,0.707725,0.191000,-0.099130,-0.554695,0.028629,-0.222195,-0.106089,-0.220430,-0.221395,-0.430205,0.115315,0.005581,0.277160,-0.078190,0.095233,0.239060,0.362955,0.112380,0.010685,0.040858,0.124330,0.119240
3,-0.263130,0.179392,-0.182415,-0.034288,0.226305,-0.286315,0.136521,-0.509905,-0.039264,1.191148,-0.420030,-0.185189,-0.334060,-0.117387,-0.346842,0.341970,0.107576,0.149692,0.257840,0.202390,-0.006959,-0.551720,-0.057687,-0.030557,0.032049,-0.183285,-0.280445,-0.016786,0.186309,-0.078932,-0.251839,-0.017106,-0.033940,0.033105,-0.034110,-0.113041,-0.103345,0.094206,0.151476,-0.071032,...,-0.010334,0.211215,-0.305863,-0.210940,-0.290445,-0.167355,-0.212200,0.243525,-0.596060,0.324757,-0.617505,0.105830,-0.215509,0.361130,-0.243903,-0.141154,0.260535,-0.271169,-0.039760,-0.099450,-0.219580,-0.110766,-0.165197,0.193366,0.357330,0.128673,0.010330,-0.016996,-0.181050,-0.597985,0.088730,-0.165042,0.129510,-0.094424,0.175135,-0.025135,-0.141150,-0.145830,-0.141230,0.133720
4,-0.035229,-0.075162,-0.328927,-0.137738,0.207256,-0.264803,0.091745,0.115009,0.020484,2.145968,-0.035718,0.215426,-0.097996,-0.314988,-0.021749,-0.036098,-0.083969,0.737270,-0.172972,-0.177402,-0.037913,-0.198370,-0.139390,-0.190470,-0.066548,-0.158304,-0.167674,-0.027081,-0.054422,-0.109294,-0.209716,-0.060278,0.015150,-0.028849,0.073894,-0.069927,-0.075503,-0.015362,-0.088853,-0.147032,...,0.024819,0.105500,-0.154887,-0.005785,0.136937,0.187975,-0.021192,0.051044,0.060776,0.211873,-0.465099,-0.016566,-0.197809,0.235915,0.046023,0.079564,-0.007832,0.065228,0.072152,-0.041293,0.070952,0.173175,-0.271408,0.028001,0.230904,0.062828,0.013761,0.034321,0.

In [22]:
#@title Use name column - VECTORIZE - Results to be df3
# Making computer sense out of all these words
then = time.time() #Time before the operations start

print("Grab a cup of coffee, this is gonna take a while.")
print("Started running:")
print(strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime()))
x = [nlp(str(item)).vector for item in df['name_artist']]
now = time.time() #Time after it finished

z = (now-then)/60
print(f"It took: {z:0.1f} minutes to vectorize name_artist column")
df3 = pd.DataFrame(x)
print(df3.shape)
df3.head()

Grab a cup of coffee, this is gonna take a while.
Started running:
Thu, 07 Jan 2021 00:40:46 +0000
It took: 17.8 minutes to vectorize name_artist column
(170654, 300)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,-0.104343,0.256664,-0.000575,-0.123706,0.105237,0.071267,0.105415,-0.218627,0.005914,0.844697,-0.221438,0.156455,0.026264,0.030606,-0.140692,0.068536,0.117969,0.855142,-0.109056,-0.001974,0.072097,0.122346,0.017775,-0.085537,0.057242,0.026935,-0.226265,0.050306,0.309934,0.060790,0.061881,-0.081949,-0.090020,0.099509,0.033985,-0.039915,0.041412,-0.007238,-0.001329,-0.035478,...,0.124067,0.065372,-0.008878,0.168204,-0.066596,-0.140428,0.105005,-0.096926,-0.031097,0.056340,-0.217942,-0.113236,-0.022624,0.002774,-0.105052,0.097090,0.131393,-0.130299,-0.072240,0.139426,0.266036,-0.163241,0.159167,-0.192002,-0.102537,-0.001253,-0.050798,-0.211696,0.099663,-0.115656,0.211048,-0.040847,-0.069745,-0.008876,0.224507,0.073500,-0.034220,-0.063754,-0.210584,0.030777
1,-0.110043,0.270508,-0.017440,-0.060651,0.316853,0.039507,-0.041189,-0.027936,0.023999,1.406347,-0.076667,-0.308565,0.289800,0.039321,-0.074606,-0.094344,-0.081386,0.721140,-0.153476,0.055845,0.069611,0.001416,0.141988,-0.147071,-0.001511,-0.116420,-0.266695,0.137878,-0.025549,0.070947,0.010973,-0.020686,0.046662,0.207328,0.106267,-0.101173,-0.115103,0.051799,0.054557,-0.006665,...,0.070192,0.008541,-0.135586,0.105572,0.007952,-0.314307,0.097951,-0.072360,-0.182626,0.002562,0.080890,-0.098343,0.094457,0.064836,0.000665,-0.155242,-0.005561,0.191624,0.077312,0.033754,-0.007187,-0.151586,-0.137674,-0.091476,0.086862,0.007816,0.043054,-0.213127,0.109522,0.044134,-0.187479,0.182865,0.123634,-0.043874,0.015906,-0.132141,-0.099888,-0.006539,-0.181266,-0.124857
2,0.068590,0.113422,0.103174,0.038538,0.106071,0.088705,0.072574,-0.155586,0.002221,0.001743,-0.118041,0.039472,0.041700,0.005835,-0.017671,0.018833,0.012381,0.150369,0.148339,0.073717,-0.019314,0.195209,0.018395,0.000544,-0.048497,-0.052766,-0.024003,-0.043881,-0.004486,0.166690,0.113390,-0.169941,-0.053533,0.020867,-0.126260,-0.112777,0.016822,0.002729,-0.153207,-0.014552,...,0.131772,-0.095836,0.076595,0.070588,-0.014139,-0.024047,0.035469,0.036178,-0.153459,0.019641,-0.275619,-0.116924,0.066542,0.062275,-0.050696,-0.000262,0.057690,0.014037,0.169320,0.062680,0.052975,-0.162852,0.066968,-0.093190,-0.019099,-0.045745,-0.047715,-0.165826,0.078896,-0.022478,0.051376,-0.017264,0.000504,0.085651,0.046241,0.034768,-0.003681,-0.013761,-0.016225,0.021574
3,-0.292194,0.387879,-0.102738,0.051408,0.057164,0.021752,0.242027,-0.276544,-0.029607,1.168303,-0.139846,-0.250231,-0.026075,-0.171859,-0.346144,0.139051,0.030024,0.482731,0.129428,0.087337,0.008702,-0.105756,0.003223,-0.240241,-0.020710,-0.064412,-0.271561,0.211825,0.224007,-0.024155,0.052121,0.193486,-0.145867,0.257438,-0.164728,-0.129346,-0.072548,0.062923,-0.073355,-0.010203,...,-0.079429,-0.026412,-0.232371,0.142913,-0.112252,-0.249662,-0.003790,0.195182,-0.475334,0.271938,-0.593094,0.148620,0.103629,0.318412,-0.116806,-0.110854,0.159057,-0.258703,-0.075874,0.031026,0.104132,-0.017178,-0.013615,0.214068,0.253996,0.014365,0.123764,-0.001096,-0.074394,-0.286461,0.212364,-0.084463,-0.022306,0.056945,0.207422,-0.061910,-0.045042,-0.193597,-0.233584,0.041322
4,-0.032911,0.133576,-0.197918,-0.142909,0.181914,-0.181740,0.051254,-0.082904,-0.087125,1.545840,-0.121138,0.068509,0.018185,-0.203634,-0.098416,-0.128725,-0.050222,0.503940,-0.142028,-0.087403,0.032555,-0.013032,-0.048948,-0.219706,-0.031451,-0.160643,-0.315007,0.002218,0.024770,-0.071185,-0.094831,-0.147687,0.010822,0.118725,0.029225,-0.071990,-0.124607,-0.045435,-0.186694,-0.057256,...,-0.074111,0.123906,-0.127728,0.188096,0.080501,-0.009474,0.008095,0.125152,-0.141503,0.232485,-0.425659,0.010340,-0.034684,0.229545,-0.018984,0.049981,0.069445,-0.022785,-0.058277,-0.111073,0.209595,0.178669,-0.202245,0.124928,0.203760,-0.032610,0.093533,-0.029013,0.1053

In [27]:
# Playing it safe and checking
print("df2 Sum of null values")
print(df2.isnull().sum())
print("="*25)
print("df3 Sum of null values")
print(df3.isnull().sum())

df2 Sum of null values
0      0
1      0
2      0
3      0
4      0
      ..
295    0
296    0
297    0
298    0
299    0
Length: 300, dtype: int64
df3 Sum of null values
0      0
1      0
2      0
3      0
4      0
      ..
295    0
296    0
297    0
298    0
299    0
Length: 300, dtype: int64


# Bring in Unit 3 model

In [28]:
dff = df
dff

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,name_artist
0,0.0594,1921,0.98200,"Sergei Rachmaninoff, James Levine, Berliner Ph...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954,"Piano Concerto No. 3 in D Minor, Op. 30: III. ..."
1,0.9630,1921,0.73200,Dennis Day,0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936,"Clancy Lowered the Boom, Dennis Day"
2,0.0394,1921,0.96100,KHP Kridhamardawa Karaton Ngayogyakarta Hadini...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339,"Gati Bali, KHP Kridhamardawa Karaton Ngayogyak..."
3,0.1650,1921,0.96700,Frank Parker,0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109,"Danny Boy, Frank Parker"
4,0.2530,1921,0.95700,Phil Regan,0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665,"When Irish Eyes Are Smiling, Phil Regan"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170649,0.7340,2020,0.20600,Ashnikko,0.717,150654,0.753,0,0OStKKAuXlxA0fMH54Qs6E,0.000000,7,0.101,-6.020,1,Halloweenie III: Seven Days,68,2020-10-23,0.0605,137.936,"Halloweenie III: Seven Days, Ashnikko"
170650,0.6370,2020,0.10100,MAMAMOO,0.634,211280,0.858,0,4BZXVFYCb76Q0Klojq4piV,0.000009,4,0.258,-2.226,0,AYA,76,2020-11-03,0.0809,91.688,"AYA, MAMAMOO"
170651,0.1950,2020,0.00998,Eminem,0.671,337147,0.623,1,5SiZJoLXp3WOl3J4C8IK0d,0.000008,2,0.643,-7.161,1,Darkness,70,2020-01-17,0.3080,75.055,"Darkness, Eminem"
170652,0.6420,2020,0.13200,"KEVVO, J Balvin",0.856,189507,0.721,1,7HmnJHfs0BkFzX4x8j0hkl,0.004710,7,0.182,-4.928,1,Billetes Azules (with J Balvin),74,2020-10-16,0.1080,94.991,"Billetes Azules (with J Balvin), KEVVO, J Balvin"


In [38]:
dff['name_artist'] = dff['name'] + ", " + dff['artists']
dff['name_artist']

0         Piano Concerto No. 3 in D Minor, Op. 30: III. ...
1                       Clancy Lowered the Boom, Dennis Day
2         Gati Bali, KHP Kridhamardawa Karaton Ngayogyak...
3                                   Danny Boy, Frank Parker
4                   When Irish Eyes Are Smiling, Phil Regan
                                ...                        
170649                Halloweenie III: Seven Days, Ashnikko
170650                                         AYA, MAMAMOO
170651                                     Darkness, Eminem
170652     Billetes Azules (with J Balvin), KEVVO, J Balvin
170653    上を向いて歩こう Ue Wo Muite Arukou Sukiyaki, Kyu Saka...
Name: name_artist, Length: 170654, dtype: object

In [39]:
f_labels = ['valence', 'year', 'acousticness', 'danceability',
       'duration_ms', 'energy', 'explicit', 'instrumentalness',
       'key', 'liveness', 'loudness', 'mode','popularity', 'speechiness', 'tempo']
scaler = MinMaxScaler()
spotify = dff.copy()
spotify[f_labels] = scaler.fit_transform(spotify[f_labels])
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=5)
spotify_matrix = spotify[f_labels].to_numpy()
model_knn.fit(spotify_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [40]:
t1_start = process_time()  
track_search = process.extractOne('Clancy Lowered the Boom', spotify['name'])

t1_stop = process_time() 
   
print("Elapsed time:", t1_stop, t1_start)
print("Elapsed time during the whole program in seconds:", 
      t1_stop-t1_start)  

Elapsed time: 2147.903245345 2138.462356472
Elapsed time during the whole program in seconds: 9.440888872999949


In [41]:
track_search

('Clancy Lowered the Boom', 100, 1)

In [42]:
def song_recomender(test, column):
    """returns indices from the model,
    pass them to df.loc[[]]"""
    id_track = process.extractOne(test,spotify[column])[2]
    print('Song selected', spotify[column][id_track], 'id:', id_track)
    print('searching for recomendation....')
    distances, ids = model_knn.kneighbors(spotify_matrix[[id_track]], n_neighbors=20)
    return ids

In [43]:
test = "gasolina"
column = "name_artist"
rec = list(song_recomender(test, column)[0])

Song selected Gasolina, Daddy Yankee id: 16437
searching for recomendation....


In [44]:
spotify.loc[rec]

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,name_artist
16437,0.753,0.838384,0.333333,Daddy Yankee,0.867409,0.034731,0.801,0.0,6jEZLz3YpnEBRpVkv35AmP,0.000001,0.000000,0.0789,0.837851,1.0,Gasolina,0.77,2004-07-13,0.063711,0.394276,"Gasolina, Daddy Yankee"
16191,0.748,0.818182,0.373494,Bob Marley & The Wailers,0.924089,0.042980,0.724,0.0,6ZXmTH4DpP07a2TvHx1EF9,0.000201,0.000000,0.0824,0.806092,1.0,Could You Be Loved,0.65,2002-01-01,0.108247,0.424341,"Could You Be Loved, Bob Marley & The Wailers"
57253,0.898,1.000000,0.351406,Anuel AA,0.853239,0.038070,0.845,0.0,7kOf0qHO4Rcwk6QJifZDRJ,0.001680,0.000000,0.0527,0.902921,1.0,El Manual,0.78,2020-05-29,0.051856,0.521500,"El Manual, Anuel AA"
54265,0.775,0.848485,0.207831,"Hector & Tito, Luny Tunes, Noriega",0.795547,0.033709,0.819,0.0,4qbi9JDziBh3cxi9SFUbeu,0.000000,0.000000,0.0969,0.860355,1.0,"Baila Morena (with Luny Tunes, Noriega) - Remix",0.63,2005-07-10,0.111340,0.386034,"Baila Morena (with Luny Tunes, Noriega) - Remi..."
16771,0.855,0.848485,0.228916,Fonseca,0.722672,0.041415,0.850,0.0,2in1gSZtv2lRbcZUmN9aAl,0.000000,0.000000,0.1080,0.816475,1.0,Te Mando Flores,0.72,2005-05-01,0.055773,0.418916,"Te Mando Flores, Fonseca"
13305,0.731,0.676768,0.276104,Grupo Niche,0.707490,0.057607,0.830,0.0,26ucR33snMkRD6JExhoD6I,0.002410,0.000000,0.1210,0.768381,1.0,Una Aventura,0.65,1988,0.037629,0.381324,"Una Aventura, Grupo Niche"
17522,0.798,0.888889,0.391566,"Jenifer Lewis, The Pinnacle Gospel Choir",0.727733,0.030162,0.825,0.0,7kh64k3P9Fk4EsA6vOdwmj,0.000004,0.000000,0.1030,0.872915,1.0,Dig a Little Deeper,0.62,2009-11-23,0.053711,0.402547,"Dig a Little Deeper, Jenifer Lewis, The Pinnac..."
16502,0.617,0.838384,0.399598,Marc Anthony,0.699393,0.054656,0.658,0.0,3yEtuhGR8jOtusa29jWrhG,0.000046,0.000000,0.1120,0.837898,1.0,Tu Amor Me Hace Bien - Salsa Version,0.70,2004-07-27,0.037010,0.401138,"Tu Amor Me Hace Bien - Salsa Version, Marc Ant..."
57319,0.887,1.000000,0.169679,Bad Bunny,0.870445,0.031641,0.791,0.0,4r9jkMEnArtWGH2rL2FZl0,0.000106,0.000000,0.1060,0.864709,1.0,A Tu Merced,0.82,2020-02-28,0.058557,0.377907,"A Tu Merced, Bad Bunny"
14862,0.816,0.757576,0.189759,Chaka Khan,0.808704,0.051079,0.878,0.0,2NVpYQqdraEcQwqT7GhUkh,0.000105,0.090909,0.1400,0.828298,1.0,Ain't Nobody,0.70,1996-11-08,0.044742,0.428259,"Ain't Nobody, Chaka Khan"


In [47]:
type(rec)

list

In [48]:
rec

[16437,
 16191,
 57253,
 54265,
 16771,
 13305,
 17522,
 16502,
 57319,
 14862,
 35317,
 18133,
 17710,
 89058,
 14848,
 56041,
 15755,
 56296,
 13890,
 11616]

In [49]:
rec = song_recomender(test, column)

Song selected Gasolina, Daddy Yankee id: 16437
searching for recomendation....


In [50]:
rec

array([[16437, 16191, 57253, 54265, 16771, 13305, 17522, 16502, 57319,
        14862, 35317, 18133, 17710, 89058, 14848, 56041, 15755, 56296,
        13890, 11616]])

# Start Playing Here!

In [76]:
#@title Get the Model Right  { run: "auto" }
#@markdown ### Algorithm:
#@markdown ## Auto will choose for you
algorithm = "ball_tree" #@param ["ball_tree", "kd_tree", "brute", "auto"]
#@markdown ### Leaf Size default is 30
#@markdown ### This can affect the speed of the construction and query, as well as the memory requierd to store the tree.  The optimal value depends on the nature of the problem.
leaf_size = 15 #@param {type:"integer"}
#@markdown ### Metric  default = minkowski
metric = "minkowski" #@param {type:"string"}
#@markdown ### Metric Parameters
metric_params = "None" #@param {type:"string"}
#@markdown ### number of jobs
n_jobs =  -1#@param {type:"integer"}
#@markdown # Nearest Neighbors
n_neighbors = 20 #@param {type:"integer"}

"""
Original Code From Yuan Jin
nn = NearestNeighbors(n_neighbors = 3, algorithm='ball_tree')
"""



nn = NearestNeighbors(algorithm=algorithm, leaf_size=leaf_size, 
                      n_neighbors=n_neighbors, n_jobs=n_jobs)
nn2 = NearestNeighbors(algorithm=algorithm, leaf_size=leaf_size, 
                      n_neighbors=n_neighbors, n_jobs=n_jobs)


nn.fit(df2)
nn2.fit(df3)


NearestNeighbors(algorithm='ball_tree', leaf_size=15, metric='minkowski',
                 metric_params=None, n_jobs=-1, n_neighbors=20, p=2,
                 radius=1.0)

In [74]:
#@title Default title text  { run: "auto" }
test = "sukiyaki" #@param {type:"string"}
model = "NLP" #@param ["NLP", "fuzzywuzzy"]
column = "name" #@param ["name", "name_artist"]




# test = "gasolina"
# column = "name_artist"
# rec = list(song_recomender(test, column)[0])




# if column == "name":
#     results = nn.kneighbors(vect)
# else:
#     results = nn2.kneighbors(vect)


test_vect = nlp(test).vector
vect = test_vect.reshape(1, -1)

if model == "NLP":
    if column == "name":
        results = nn.kneighbors(vect)[1].tolist()[0]
        print("Using song title as search")
    else:
        results = nn2.kneighbors(vect)[1].tolist()[0]
        print("Using song title and artist as search")
else:
    results = song_recomender(test, column)[0].tolist()


s = list([df['name'].iloc[x] for x in results])
a = [df['artists'].iloc[x] for x in results]
y = [df['release_date'].iloc[x] for x in results]
     

# print(songs)
# print(results[1][0])
cn = ["Song", "Artist", "Year", "Row #"]
print(f"{cn[0]:<51}{cn[1]:<45}{cn[2]:<15}{cn[3]:^25}")
print("=" * 126)
for i in range(n_neighbors-1):
    print(f"{s[i][0:50]:<51}{a[i][0:44]:<45}{y[i]:^15}{results[i]:>16}")

Using song title as search
Song                                               Artist                                       Year                     Row #          
Sukiyaki                                           ['4 P.M.']                                     1995-01-01             103709
Sukiyaki                                           ['Kyu Sakamoto']                               2010-01-01             168549
Sukiyaki                                           ['Selena']                                     1993-01-01              86857
Sukiyaki - Remastered                              ['A Taste Of Honey']                           2002-01-01             166862
Hay Nabe Nabe                                      ['Seyed Ali Asghar Kurdistani']                1930-03-02              20862
上を向いて歩こう Ue Wo Muite Arukou Sukiyaki               ['Kyu Sakamoto']                               2010-01-01             170653
Soju Hanjan                                        ['Lim Changjung']

In [66]:
#@title Default title text  { run: "auto" }
test = "\u4E0A\u3092\u5411\u3044\u3066\u6B69\u3053\u3046" #@param {type:"string"}
model = "fuzzywuzzy" #@param ["NLP", "fuzzywuzzy"]
column = "name_artist" #@param ["name", "name_artist"]




# test = "gasolina"
# column = "name_artist"
# rec = list(song_recomender(test, column)[0])




# if column == "name":
#     results = nn.kneighbors(vect)
# else:
#     results = nn2.kneighbors(vect)


test_vect = nlp(test).vector
vect = test_vect.reshape(1, -1)

if model == "NLP":
    if column == "name":
        results = nn.kneighbors(vect)[1].tolist()[0]
        print("Using song title as search")
    else:
        results = nn2.kneighbors(vect)[1].tolist()[0]
        print("Using song title and artist as search")
else:
    results = song_recomender(test, column)[0].tolist()


s = list([df['name'].iloc[x] for x in results])
a = [df['artists'].iloc[x] for x in results]
y = [df['release_date'].iloc[x] for x in results]
     

# print(songs)
# print(results[1][0])
cn = ["Song", "Artist", "Year", "Row #"]
print(f"{cn[0]:<51}{cn[1]:<45}{cn[2]:<15}{cn[3]:^25}")
print("=" * 126)
for i in range(n_neighbors-1):
    print(f"{s[i][0:50]:<51}{a[i][0:44]:<45}{y[i]:^15}{results[i]:>16}")

Song selected 上を向いて歩こう Ue Wo Muite Arukou Sukiyaki, Kyu Sakamoto id: 170653
searching for recomendation....
Song                                               Artist                                       Year                     Row #          
上を向いて歩こう Ue Wo Muite Arukou Sukiyaki               ['Kyu Sakamoto']                               2010-01-01             170653
Working Class Hero - Remastered 2010               ['John Lennon']                                1970-12-11               9784
Heart In A Cage                                    ['Chris Thile']                                2006-01-01              72330
Wiseman                                            ['Slightly Stoopid']                           2001-03-16              34748
With You                                           ['Lil Wayne', 'Drake']                         2010-01-01             106782
Все идет по плану                                  ['Egor Letov']                                    1988          

# Web App Starts Here

In [75]:
#@title FastAPI Starts Here

cmd = 'python -m http.server 9999'
ColabCustom(cmd)

Web App can be accessed on: http://0ae01aa40721.ngrok.io


KeyboardInterrupt: ignored